<a href="https://colab.research.google.com/github/Gafoor448/base/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [7]:
# Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/BrainTumor/TRAIN',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/BrainTumor/TRAIN',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/BrainTumor/TEXT',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2416 images belonging to 2 classes.
Found 604 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
76/76 [==============================] - 318s 4s/step - loss: 0.4836 - accuracy: 0.7711 - val_loss: 0.3047 - val_accuracy: 0.8659
Epoch 2/5
76/76 [==============================] - 304s 4s/step - loss: 0.2125 - accuracy: 0.9151 - val_loss: 0.1277 - val_accuracy: 0.9603
Epoch 3/5
76/76 [==============================] - 287s 4s/step - loss: 0.0928 - accuracy: 0.9719 - val_loss: 0.0708 - val_accuracy: 0.9768
Epoch 4/5
76/76 [==============================] - 285s 4s/step - loss: 0.0507 - accuracy: 0.9830 - val_loss: 0.1338 - val_accuracy: 0.9503
Epoch 5/5
76/76 [==============================] - 287s 4s/step - loss: 0.0328 - accuracy: 0.9888 - val_loss: 0.0114 - val_accuracy: 0.9983


In [13]:
model.save("Model.h5","label.txt")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from keras.preprocessing import image
import numpy as np

#load the model
model = load_model('/content/Model.h5')
import numpy as np
from keras.preprocessing import image

test_image_path='/content/drive/MyDrive/Colab Notebooks/BrainTumor/TEXT/pred/pred10.jpg'

# Load and preprocess the test image
img = image.load_img(test_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize the pixel values

prediction = model.predict(img_array)
if prediction < 0.5 :
  print("prediction : No tumor (probability:",prediction[0][0], ")")
else :
  print("prediction : Tumor present (probability :",prediction[0][0], " )")

1/1 [==============================] - 0s 151ms/step
prediction : Tumor present (probability : 0.9998674  )


In [14]:
#test your image

from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
model = load_model('/content/Model.h5')
class_names = ['You have Brain Tumor','You do not have Brain Tumor']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)


In [15]:
image = Image.open("/content/drive/MyDrive/Colab Notebooks/BrainTumor/TEXT/pred/pred37.jpg").convert("RGB")
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
image_array = np.asarray(image)
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
data[0] = normalized_image_array


In [16]:
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)


1/1 [==============================] - 0s 134ms/step
Result:  You have Brain Tumor

Accuracy:  0.49495745
